# QUANT ANALYSIS

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd


In [ ]:
ticker = "TSLA"
benchmark = "SPY"
start_date = "2022-01-01"
end_date = "2025-07-01"
z_threshold = 2.5
vol_window = 10        # Rolling window for volatility (10 days)
vol_multiplier = 2.0   # Spike if return > 2 × rolling volatility

In [ ]:
data = yf.download(ticker, start=start_date, end=end_date)
benchmark_data = yf.download(benchmark, start=start_date, end=end_date)

/tmp/ipython-input-5-3835138765.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-5-3835138765.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  benchmark_data = yf.download(benchmark, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [ ]:
data['Return'] = data['Close'].pct_change()
benchmark_data['Benchmark_Return'] = benchmark_data['Close'].pct_change()
benchmark_returns = benchmark_data[['Benchmark_Return']]
data = data.join(benchmark_returns, how='left')
data['Relative_Return'] = data['Return'] - data['Benchmark_Return']

In [ ]:
#Z_score and Deviation
mean_ret = data['Relative_Return'].mean()
std_ret = data['Relative_Return'].std()
data['Z_score'] = (data['Relative_Return'] - mean_ret) / std_ret
data['Z_Event'] = np.where(abs(data['Z_score']) > z_threshold,
                           np.where(data['Z_score'] > 0, 'Positive Outlier', 'Negative Outlier'),
                           None)

In [ ]:
#Volatility
data['Rolling_STD'] = data['Relative_Return'].rolling(window=vol_window).std()
data['Vol_Spike'] = np.where(abs(data['Relative_Return']) > vol_multiplier * data['Rolling_STD'],
                              'Volatility Spike', None)

In [ ]:
events = data[(data['Z_Event'].notna()) | (data['Vol_Spike'].notna())].copy()
events['Event_Type'] = events[['Z_Event', 'Vol_Spike']].fillna('').agg(', '.join, axis=1).str.strip(', ')
events['Ticker'] = ticker
events = events.reset_index()[['Date', 'Ticker', 'Return', 'Z_score', 'Rolling_STD', 'Event_Type']]

In [ ]:
print("Combined Market Events:")
print(events.tail(10))

# RETRIVAL

In [ ]:
!pip install yahoo_fin


In [ ]:
from yahoo_fin import news
from datetime import timedelta
from datetime import datetime
import requests
from google.colab import userdata

In [ ]:
all_news = news.get_yf_rss(ticker)
news_df = pd.DataFrame(all_news)
news_df['date'] = news_df['published_parsed'].apply(lambda x: datetime(*x[:6]))

In [ ]:
news_df['date_only'] = news_df['date'].dt.date
events['date_only'] = events['Date'].dt.date

In [ ]:
#FINHUB_API
FINNHUB_API_KEY = userdata.get('NEWS')

def get_historical_news(ticker, date, window_days=1):
    start = (date - timedelta(days=window_days)).strftime('%Y-%m-%d')
    end = (date + timedelta(days=window_days)).strftime('%Y-%m-%d')
    url = f"https://finnhub.io/api/v1/company-news?symbol={ticker}&from={start}&to={end}&token={FINNHUB_API_KEY}"
    r = requests.get(url)
    articles = r.json()
    if isinstance(articles, list):
        return '; '.join([a['headline'] for a in articles[:5]])  # Top 5
    return ''


In [ ]:
events['News_Headlines'] = events['Date'].apply(lambda d: get_historical_news(ticker, d))


In [ ]:
events = events[events['News_Headlines'].notna() & (events['News_Headlines'].str.strip() != '')].copy()

In [ ]:

print("Final Market Events with News:")
print(events[['Date', 'Event_Type', 'Z_score', 'Return', 'News_Headlines']])

Final Market Events with News:
Price        Date                          Event_Type   Z_score    Return  \
Ticker                                                                      
44     2024-10-11  Negative Outlier, Volatility Spike -2.728466 -0.087825   
45     2024-10-24  Positive Outlier, Volatility Spike  6.300916  0.219190   
46     2024-11-06                    Positive Outlier  3.559259  0.147510   
47     2024-11-11                    Positive Outlier  2.571573  0.089596   
48     2024-12-11                    Volatility Spike  1.494738  0.059303   
49     2024-12-16                    Volatility Spike  1.656539  0.061413   
50     2025-02-03                    Volatility Spike -1.309825 -0.051705   
51     2025-02-11                    Volatility Spike -1.866590 -0.063382   
52     2025-02-25                    Volatility Spike -2.295913 -0.083896   
53     2025-03-10  Negative Outlier, Volatility Spike -3.710683 -0.154262   
54     2025-03-24  Positive Outlier, Volatili

In [ ]:
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Only compute for rows with headlines
events = events.copy()
events['News_Headlines'] = events['News_Headlines'].fillna('')

# Compute embeddings
embeddings = model.encode(events['News_Headlines'].tolist(), show_progress_bar=True)

# Store as separate column (can be saved later to disk or FAISS)
import numpy as np
events['Embedding'] = embeddings.tolist()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.6 MB/s eta 0:00:00


In [ ]:
import numpy as np

embedding_matrix = np.array(events['Embedding'].tolist()).astype('float32')


In [ ]:
dimension = embedding_matrix.shape[1]  # 384
index = faiss.IndexFlatL2(dimension)
index.add(embedding_matrix)


In [ ]:
query_vector = embedding_matrix[0].reshape(1, -1)

# k nearest neighbors
k = 5
D, I = index.search(query_vector, k)  # D: distances, I: indices

# Show similar events
print("Most similar events to Event 0:")
similar_events = events.iloc[I[0]]
print(similar_events[['Date', 'Event_Type', 'Z_score', 'News_Headlines']])


In [ ]:
!pip install openai


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI



In [ ]:
from groq import Groq

In [ ]:
client = Groq(
    api_key = userdata.get("GROQ")
)

def explain_market_event(event_index, top_k=5):
    embedding = events.iloc[event_index]['Embedding']

    if isinstance(embedding[0], list):
        embedding = embedding[0]

    # Convert to float32 vector
    query_vector = np.array(embedding, dtype='float32').reshape(1, -1)

    D, I = index.search(query_vector, k=top_k)
    similar = events.iloc[I[0]]

    context = "\n\n".join([
        f"Date: {events['Date']}, Event: {events['Event_Type']}, News: {events['News_Headlines']}"
        for _, row in similar_events.iterrows()
    ])

    prompt = f"""You are a financial analyst assistant.
Based on the following historical news and events similar to today's anomaly, explain what might be causing this market movement.

Similar Events:
{context}

Today's Event: {events.iloc[event_index]['Event_Type']}, News: {events.iloc[event_index]['News_Headlines']}

Explanation:"""



    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
     )

    return response.choices[0].message.content


In [ ]:
explanation = explain_market_event(event_index=len(events) - 2)
print(explanation)


/tmp/ipython-input-112-929677601.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(embedding[0], list):
/tmp/ipython-input-112-929677601.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  embedding = embedding[0]


Based on the historical data provided, today's market movement appears to be a Negative Outlier with a Volatility Spike. This event is similar to others in the past, where a significant departure of a key executive or a major announcement related to Tesla has led to increased market volatility.

In this case, the news headline "Departure of Tesla’s humanoid robot boss caps" suggests that a key executive responsible for Tesla's humanoid robot project has left the company. This departure may have raised concerns among investors about the project's future prospects and potentially impacted Tesla's stock price.

Given the historical context, it is likely that the market movement today is driven by:

1. **Investor uncertainty**: The departure of a key executive can create uncertainty about the company's future plans and strategy, leading to increased volatility.
2. **Tesla's influence on the market**: As a major player in the tech and automotive industries, Tesla's news and events can have 